In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from matplotlib.ticker import MaxNLocator
import json
import ptitprince as pt
import pickle as pkl
# import contextily as ctx
from analysis import *
from ema_workbench import load_results

In [11]:
# Load results
results = load_results('../results/scenarios=150, policies=16.tar.gz')

In [12]:
outcomes = prepare_outcomes_dataframe(results)
experiments = results[0]

In [13]:
outcomes.drop('years_in_poverty', axis=1, inplace=True)

In [14]:
# Convert columns to float
for col in outcomes.columns:
    if col not in ['scenario', 'policy', 'district']:
        outcomes[col] = outcomes[col].astype(float)

In [15]:
policy_columns = ['target_group', 'top_up']
outcomes = pd.merge(experiments[['policy'] + policy_columns], outcomes, on='policy')
outcomes = outcomes.drop(columns=['policy', 'scenario'])

In [16]:
target_groups = outcomes['target_group'].unique().tolist()
top_ups = outcomes['top_up'].unique().tolist()
target_groups, top_ups

(['poor', 'poor_near_poor1.25', 'all', 'poor_near_poor2.0'], [50, 30, 0, 10])

In [17]:
outcomes_of_interest = ['annual_average_consumption_loss_pct', 'n_new_poor']
outcomes.groupby(['district', 'target_group', 'top_up']).mean()[outcomes_of_interest][:25]

annual_average_consumption_loss_pct  \
district           target_group       top_up                                        
AnseLaRayeCanaries all                0                                  0.208422   
                                      10                                 0.012605   
                                      30                                 0.000069   
                                      50                                      NaN   
                   poor               0                                  0.208422   
                                      10                                 0.171520   
                                      30                                      NaN   
                                      50                                 0.169592   
                   poor_near_poor1.25 0                                  0.208422   
                                      10                                 0.205652   
                                      30                                      NaN   
                                      50                                 0.205515   
                   poor_near_poor2.0  0                                       NaN   
                                      10                                 0.206466   
                                      30                                 0.206329   
                                      50                                 0.206329   
Castries           all                0                                  0.245056   
                                      10                                 0.001654   
                                      30                                 0.000073   
                                      50                                      NaN   
                   poor               0                                  0.245056   
                                      10                                 0.229864   
                                      30                                      NaN   
                                      50                                 0.229754   
                   poor_near_poor1.25 0                                  0.245056   

                                              n_new_poor  
district           target_group       top_up              
AnseLaRayeCanaries all                0        86.893333  
                                      10        3.033333  
                                      30        0.000000  
                                      50             NaN  
                   poor               0        86.893333  
                                      10       86.893333  
                                      30             NaN  
                                      50       86.893333  
                   poor_near_poor1.25 0        86.893333  
                                      10       86.893333  
                                      30             NaN  
                                      50       86.893333  
                   poor_near_poor2.0  0              NaN  
                                      10       86.893333  
                                      30       86.893333  
                                      50       86.893333  
Castries           all                0       525.493333  
                                      10        3.926667  
                                      30        0.413333  
                                      50             NaN  
                   poor               0       525.493333  
                                      10      525.493333  
                                      30             NaN  
                                      50      525.493333  
                   poor_near_poor1.25 0       525.493333

In [19]:
outcomes_of_interest = ['annual_average_consumption_loss_pct', 'n_new_poor']
outcomes.drop('district', axis=1).groupby(['target_group', 'top_up']).mean()[outcomes_of_interest]

annual_average_consumption_loss_pct  n_new_poor
target_group       top_up                                                 
all                0                                  0.262459  156.877778
                   10                                 0.007259    2.100741
                   30                                 0.000227    0.105185
                   50                                      NaN         NaN
poor               0                                  0.262459  156.877778
                   10                                 0.237432  156.877778
                   30                                      NaN         NaN
                   50                                 0.236768  156.877778
poor_near_poor1.25 0                                  0.262459  156.877778
                   10                                 0.211726  156.126667
                   30                                      NaN         NaN
                   50                                 0.209575  156.126667
poor_near_poor2.0  0                                       NaN         NaN
                   10                                 0.245926  156.877778
                   30                                 0.245196  156.877778
                   50                                 0.245196  156.877778

In [ ]:
vertical = ['None', '10%', '30%', '50%']
horizontal = ['All', 'Poor', 'Poor & Near Poor (1.25)', 'Poor & Near Poor (2.0)']
columns = ['Vertical expansion', 'Horizontal expansion', 
           'Average reduction in consumption loss (%)', 
           'Average number of people prevented from falling into poverty']